In [1]:
import json
from twython import Twython
import pandas as pd
import numpy as np


# Enter your keys/secrets as strings in the following fields
#credentials = {}
#credentials['CONSUMER_KEY'] = ''
#credentials['CONSUMER_SECRET'] = ''
#credentials['ACCESS_TOKEN'] = '-'
#credentials['ACCESS_SECRET'] = ''

# Save the credentials object to file
#with open("twitter_credentials.json", "w") as file:
#    json.dump(credentials, file)
    
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file) 
    
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])    

## look for key word

In [ ]:
query = {'q': 'energy',
        'result_type': 'recent',  # other options 'mixed','popular','recent'
        'count': 10,   # max 100
         # 'until':"2019-02-01",
         'geocode': '51.51753,-0.11214,1000mi'
        }

dict_ = {'user': [],
         'date': [],
         'text': [],
         'favorite_count': [],
         'retweet_count': [],
         'user_loc': [],
         'followers_count': [],
         'lang': [],
         'user_desc': [],
         'hashtags': [],
         'user_verified': [],
         'geo': [],
         'coordinates':[],
         'place':[],
         #'user_loc_country':[]
         
        }


for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_loc'].append( status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['lang'].append(status['lang'])
    dict_['user_desc'].append(status['user']['description'])
    dict_['user_verified'].append(status['user']['verified'])
    dict_['hashtags'].append(status['entities']['hashtags'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['geo'].append(status['geo'])
    dict_['coordinates'].append(status['coordinates'])
    dict_['place'].append(status['place'])
    #dict_['user_loc_country'].append(status['user']['derived']['locations']['country'])
    
df = pd.DataFrame(dict_)
#dict_['text'][0]
df

In [ ]:
df.shape

In [ ]:
df

In [ ]:
#cities = pd.read_csv('list_cities.csv',sep=';')

## Translate queries and Tweets

In [ ]:
from os import environ

from google.cloud import translate

def gtrans(text,dest='en'):
    
    '''this function represents the google translate API. use wisely!
    its expensive (20$/1mio characters, makes only 5000 tweets)'''
    
    
    project_id = 'lewagon-bootcamp-timwolfram' #environ.get("PROJECT_ID", "")

    parent = f"projects/{project_id}"
    client = translate.TranslationServiceClient()


    sample_text = text
    target_language_code = dest

    response = client.translate_text(
        contents=[sample_text],
        target_language_code=target_language_code,
        parent=parent,
    )

    for translation in response.translations:
        output_trans = translation.translated_text
        
    return output_trans

In [ ]:
gtrans('Hallo Welt')

In [ ]:
dict_ = {'user': [],
             'date': [],
             'text': [],
             'text_en': [],
             'favorite_count': [],
             'retweet_count': [],
             'user_loc': [],
             'followers_count': [],
             'lang': [],
             'user_desc': [],
             'user_desc_en': [],
             'hashtags': [],
             'user_verified': [],

             #'geo': [],
             #'coordinates':[],
             #'place':[]

             #'user_loc_country':[]

            }



query_word = 'energy'

lang_list = ['en','de','fr','el','it','es',
                'pl', 'ro', 'nl', 'hu','pt',
                'sv',  'cs', 'bg', 'sk', 'da',
                'fi', 'hr', 'lt'
            ]

count_tweets = 1
result_type = 'mixed'

for lang in lang_list:
    
    query_word = gtrans(query_word, dest=lang)
    if lang == 'en':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': '51.51753,-0.11214,1000mi'
            }
        
    elif lang == 'fr':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': '47.22283,2.07099,1000mi'
            }
    elif lang == 'es':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': ' 40.42955,-3.67930,1000mi'
            }
        
    elif lang == 'pt':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': '39.82059,-7.49342,1000mi'
            }
        
    else:
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
             # 'until':"2019-02-01",
             #'geocode': '50.0598058,14.3255426,1000km'
            }

   
    for status in python_tweets.search(**query)['statuses']:
        
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['text'].append(status['text'])
        dict_['text_en'].append(gtrans(str(status['text']), dest='en'))#src=str(status['lang'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['user_loc'].append( status['user']['location'])
        dict_['followers_count'].append(status['user']['followers_count'])
        dict_['lang'].append(status['lang'])
        dict_['user_desc'].append(status['user']['description'])
        try:
            dict_['user_desc_en'].append(gtrans(str(status['user']['description']), dest='en'))
        except:
            dict_['user_desc_en'].append('none')
        dict_['user_verified'].append(status['user']['verified'])
        dict_['hashtags'].append(status['entities']['hashtags'])
        dict_['retweet_count'].append(status['retweet_count'])


        #dict_['geo'].append(status['geo'])
        #dict_['coordinates'].append(status['coordinates'])
        #dict_['place'].append(status['place'])
        #dict_['user_loc_country'].append(status['user']['derived']['locations']['country'])


df = pd.DataFrame(dict_)
#dict_['text']
df


In [ ]:
#cities = pd.read_csv('list_cities2.csv', delimiter=';')

In [ ]:
#cities

In [ ]:
### filter for european countries

## sentiment analysis

In [ ]:
from textblob import TextBlob
import nltk
import string
#nltk.download('brown')
import string

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [ ]:
text = df['text_en'][1]
text

In [ ]:
#text = text.lower()

##remove punctiation

for punctuation in string.punctuation:
    text = text.replace(punctuation, '') 
    
##remove stopwords
stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(text) 
  
text = [w for w in word_tokens if not w in stop_words] 

#text = ' '.join(text)
  

In [ ]:
text = ' '.join(text)
text

In [ ]:
wiki = TextBlob(text)
#wiki.tags

In [ ]:
print(wiki.noun_phrases)

In [ ]:
testimonial = TextBlob(text)
print(testimonial.sentiment)
print(testimonial.sentiment.polarity)

## all functions for .py file

In [ ]:
from os import environ
import json
import pandas as pd
from google.cloud import translate
from twython import Twython


# Enter your keys/secrets as strings in the following fields
#credentials = {}
#credentials['CONSUMER_KEY'] = ''
#credentials['CONSUMER_SECRET'] = ''
#credentials['ACCESS_TOKEN'] = '-'
#credentials['ACCESS_SECRET'] = ''

#Save the credentials object to file
#with open("twitter_credentials.json", "w") as file:
#    json.dump(credentials, file)
    
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file) 
    
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])    


def get_places():
    
    '''creates a list of places and countries in europe, for locality-filtering the output tweets'''
    cities = pd.read_csv('list_cities3.csv',delimiter=';')
    
    list_cities = list(cities['city'])
    list_cities = [element.lower() for element in list_cities]
    list_countries = list(set(cities['county']))
    list_countries = [element.lower() for element in list_countries]
    list_eur = list_cities + list_countries
    return list_eur


def gtrans(text,dest='en'):
    
    '''this function represents the google translate API. use wisely!
    its expensive (20$/1mio characters, makes only 5000 tweets)'''
    
    
    project_id = 'lewagon-bootcamp-timwolfram' #environ.get("PROJECT_ID", "")

    parent = f"projects/{project_id}"
    client = translate.TranslationServiceClient()


    sample_text = text
    target_language_code = dest

    response = client.translate_text(
        contents=[sample_text],
        target_language_code=target_language_code,
        parent=parent,
    )

    for translation in response.translations:
        output_trans = translation.translated_text
        
    return output_trans


def local_filter(tweets, list_eur=get_places(), no_loc=True):
    
    '''filters the output of the get_tweets function to only get european posts'''
    
    if no_loc == True:
        list_eur.append('')
        
    for tweet in tweets:
        if not tweet['user_loc'].split(',')[0] in list_eur:
            tweets.remove(tweet)
        
        
    return tweets


def remove_duplicates(tweets):
    li = []
    for tweet in tweets:
        if not tweet in li:
            li.append(tweet)

    return li


def get_tweets(query,count,result_type='mixed'):   #count: max 10 because its looking for all 19 languages at once

    '''this function uses the twitter search endpoint with max 190 requests/15 min
    the output is a list of dictionaries with one dict per tweet.
    keys are the features of the tweets'''
    
    ''' it uses the google-translate API wich kosts 20$ per ~5000 tweets '''

    query_word = query
    count_tweets = count
    
    lang_list = ['en','de','fr','el','it','es',
                    'pl', 'ro', 'nl', 'hu','pt',
                    'sv',  'cs', 'bg', 'sk', 'da',
                    'fi', 'hr', 'lt'
                ]

    #dict_ = {}
    
    result_type = result_type # other options 'mixed','popular','recent'

    list_tweets = []

    for lang in lang_list:

        query_word = gtrans(query_word, dest=lang)
        if lang == 'en':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '51.51753,-0.11214,1000mi',
                'tweet_mode': 'extended'
                }

        elif lang == 'fr':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '47.22283,2.07099,1000mi',
                'tweet_mode': 'extended'
                }
            
        elif lang == 'es':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '40.42955,-3.67930,1000mi',
                'tweet_mode': 'extended'
                }

        elif lang == 'pt':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '39.82059,-7.49342,1000mi',
                'tweet_mode': 'extended'
                }

        else:
            query = {'q': query_word,
                'result_type': result_type,  # 
                'count': count_tweets,   # max 100
                #'geocode': '50.0598058,14.3255426,1000km'
                'tweet_mode': 'extended'
                }


        for status in python_tweets.search(**query)['statuses']:
            dict_ = {}
            dict_['id'] = status['id']
            dict_['user'] = status['user']['screen_name']
            dict_['date'] = status['created_at']
            dict_['text'] = status['full_text']
            dict_['text_en'] = gtrans(str(status['full_text']), dest='en')
            dict_['favorite_count'] = status['favorite_count']
            dict_['user_loc'] =  status['user']['location']
            dict_['followers_count'] = status['user']['followers_count']
            dict_['lang'] = status['lang']
            dict_['user_desc'] = status['user']['description']
            try:
                dict_['user_desc_en'] = gtrans(str(status['user']['description']), dest='en')
            except:
                dict_['user_desc_en'] = 'none'
            dict_['user_verified'] = status['user']['verified']
            dict_['hashtags'] = status['entities']['hashtags']
            dict_['retweet_count'] = status['retweet_count']

            list_tweets.append(dict_) 
    
    return list_tweets




In [ ]:
from geopy import geocoders
from geopy.geocoders import Nominatim
import math


def get_country(city):
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    country = df_europe.loc[df_europe['city'] == city, 'country'].iloc[0]
    return country


def region_lat(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return latitude


def region_lon(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return longitude


def count_cities(tweets):
    
    #in the user locations sometimes its Paris, France, sometimes France
    # sometimes Paris, here we try to filter this for cities
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    
    city_counts = {
    
    'city': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[0] in list_cities:
                city_counts['city'].append(list_loc[0])
                city_counts['likes'].append(tweet['favorite_count'])
                city_counts['tweets'].append(1)
                city_counts['retweets'].append(tweet['retweet_count'])
                city_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_cities:
            city_counts['city'].append(tweet['user_loc'])
            city_counts['likes'].append(tweet['favorite_count'])
            city_counts['tweets'].append(1)
            city_counts['retweets'].append(tweet['retweet_count'])
            city_counts['sentiment'].append(tweet['compound_score'])
            
            
            
    return city_counts


    
def count_countries(tweets):
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    country_counts = {
    
    'country': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[1] in list_countries:
                country_counts['country'].append(list_loc[1])
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])
                
            if list_loc[0] in list_cities:
                country_counts['country'].append(get_country(list_loc[0])) #translates city to country
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])

            
            
        elif tweet['user_loc'] in list_cities:
            country_counts['country'].append(get_country(tweet['user_loc']))
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_countries:
            country_counts['country'].append(tweet['user_loc'])
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
    return country_counts


def clean_and_sum(df):
    df = df.dropna()
    df = df.groupby(by="country", as_index=False).sum()
    return df


def add_lat_lon(df):
    df['lat'] = df['country'].apply(region_lat)
    df['lon'] = df['country'].apply(region_lon)
    return df


def add_radius(df):
    df["radius"] = df["retweets"].apply(lambda likes: math.sqrt(likes)*1000 + 10000)
    return df


##### define colors from sentiment ####

def sent_ref(df):
    df['sent_ref'] = df['sentiment'] / df['tweets']
    return df

def sent_color_green(sent):
    
    if sent > 0:
        green = 100
    else:
        green = 0
        
    return green


def sent_color_red(sent):
    
    if sent < 0:
        red = 100
    else:
        red = 0
    
    return red
    
    
def sent_color_blue(sent):
    blue = sent * 0
    return blue


def sent_shade(sent):
    if sent < 0:
        shade = -300 * sent + 50
    else:
        shade = 300 * sent + 50
    return shade
    
def sent_colors(df):
    df['r'] = df['sent_ref'].apply(sent_color_red)
    df['g'] = df['sent_ref'].apply(sent_color_green)
    df['b'] = df['sent_ref'].apply(sent_color_blue)
    df['s'] = df['sent_ref'].apply(sent_shade)
    df['sent_ref'] = df['sent_ref'].round(decimals=4)
    return df


In [ ]:

from geopy import geocoders
from geopy.geocoders import Nominatim

df_eur = pd.read_csv('list_cities.csv', delimiter=';')

list_countries = list(df_eur['country'])






def region_lat(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return latitude

def region_lon(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return longitude



def count_regions(tweets):
    
    #creates a dictionary with where the tweets are assigned to country and region
    #of the user-location
    
    
    region_counts = {
    'country': [],
    'city': [],
    'tweets':[],
    'likes': []
    }
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            region_counts['city'].append(list_loc[0])
            region_counts['country'].append(list_loc[1])
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
        
        elif tweet['user_loc'] in list_countries:
            region_counts['city'].append('NaN')
            region_counts['country'].append('user_loc')
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
        
        elif tweet['user_loc'] in list_cities:
            region_counts['city'].append('user_loc')
            region_counts['country'].append('NaN')
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
                
        else:
            region_counts['city'].append('NaN')
            region_counts['country'].append('NaN')
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
            
        
            
        
    return region_counts
            


    
    

## geographical mapping of tweets

In [140]:
#list_cities = ['Berlin', 'Madrid', 'Rome', 'Bucharest', 'Paris', 'Vienna', 'Hamburg', 'Warsaw', 'Budapest', 'Barcelona', 'Munich', 'Milan', 'Prague', 'Sofia', 'Cologne', 'Stockholm', 'Naples', 'Turin', 'Amsterdam', 'Marseille', 'Zagreb', 'Copenhagen', 'Valencia', 'Krakow', 'Frankfurt', 'Seville', 'Lodz', 'Zaragoza', 'Athens', 'Palermo', 'Helsinki', 'Rotterdam', 'Wroc?aw', 'Stuttgart', 'Riga', 'Düsseldorf', 'Vilnius', 'Leipzig', 'Dortmund', 'Essen', 'Gothenburg', 'Genoa', 'Malaga', 'Bremen', 'Dresden', 'Dublin', 'The Hague', 'Hanover', 'Poznan', 'Antwerp', 'Nuremberg', 'Lyon', 'Lisbon', 'Duisburg', 'Toulouse', 'Gdansk', 'Murcia', 'Tallinn', 'Bratislava', 'Palma de Mallorca', 'Szczecin', 'Bologna', 'Brno', 'Ia?i', 'Florence', 'Las Palmas', 'Bochum', 'Utrecht', 'Wuppertal', 'Aarhus', 'Bydgoszcz', 'Plovdiv', 'Bilbao', 'Malmö', 'Nice', 'Lublin', 'Varna', 'Bielefeld', 'Alicante', 'Timi?oara', 'Bonn', 'Cordoba', 'Thessaloniki', 'Cluj-Napoca', 'Bari', 'Constanza', 'Münster', 'Karlsruhe', 'Catania', 'Mannheim', 'Nantes', 'Craiova', 'Galati']
df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
list_cities = list(df_europe['city'])
list_countries = list(df_europe['country'])

In [132]:
print(list_cities)

['MOSKVA ', 'LONDON', 'St Petersburg', 'BERLIN', 'MADRID', 'ROMA', 'KIEV', 'PARIS', 'BUCURESTI ', 'BUDAPEST', 'Hamburg', 'MINSK', 'WARSZAWA ', 'BEOGRAD (Belgrade)', 'WIEN ', 'Kharkov', 'Barcelona', 'Novosibirsk', 'Nizhny Novgorod', 'Milano (Milan)', 'Ekaterinoburg', 'M�nchen ', 'PRAHA ', 'Samara', 'Omsk', 'SOFIA', 'Dnepropetrovsk', 'Kazan', 'Ufa', 'Chelyabinsk', 'Donetsk ', 'Napoli ', 'Birmingham', 'Perm', 'Rostov-na-Donu', 'Odessa', 'Volgograd', 'K�ln ', 'Torino ', 'Voronezh', 'Krasnoyarsk', 'Saratov', 'ZAGREB', 'Zaporozhye', 'L�dz', 'Marseille', 'RIGA', 'Lvov', 'ATHINAI ', 'Salonika', 'STOCKHOLM', 'Krak�w', 'Valencia', 'AMSTERDAM', 'Leeds', 'Tolyatti', 'Kryvy Rig', 'Sevilla', 'Palermo', 'Ulyanovsk', 'KISHINEV', 'Genova', 'Izhevsk', 'Frankfurt am Main', 'Krasnodar', 'Wroclaw ', 'Glasgow', 'Yaroslave', 'Khabarovsk', 'Vladivostok', 'Zaragoza', 'Essen', 'Rotterdam', 'Irkutsk', 'Dortmund', 'Stuttgart', 'Barnaul', 'VILNIUS', 'Poznan', 'D�sseldorf', 'Novokuznetsk', 'LISBOA ', 'HELSINKI', 'M

In [133]:

#print(list_cities)
print('Lisbon' in list_cities)

True


In [137]:
def get_country(city):
    country = df_europe.loc[df_europe['city'] == city, 'country'].iloc[0]
    return country

In [139]:
print(get_country('Cologne'))

Germany


In [ ]:
def get_cities

In [196]:
from geopy import geocoders
from geopy.geocoders import Nominatim


    #this cell is about generating the geolocation information of the tweets
    #and sorting the features we need for the map display
    
     
df_eur = pd.read_csv('list_cities.csv', delimiter=';')

list_countries = list(df_eur['country'])

def get_country(city):
    
    #translates city-name to country name
    
    country = df_europe.loc[df_europe['city'] == city, 'country'].iloc[0]
    return country



def city_lat_long(city):
    geolocator = Nominatim(user_agent='myapplication')
    location = geolocator.geocode(city)
    latitude = location[1][0]
    longitude = location[1][1]
    return latitude, longitude


def region_lat(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return latitude

def region_lon(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return longitude



def count_regions(tweets):
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    region_counts = {
    'country': [],
    'city': [],
    'tweets':[],
    'likes': []
    }
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            region_counts['city'].append(list_loc[0])
            region_counts['country'].append(list_loc[1])
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
        
        elif tweet['user_loc'] in list_countries:
            region_counts['city'].append('NaN')
            region_counts['country'].append('user_loc')
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
        
        elif tweet['user_loc'] in list_cities:
            region_counts['city'].append(tweet['user_loc'])
            region_counts['country'].append('NaN')
            region_counts['likes'].append(tweet['favorite_count'])
            region_counts['tweets'].append(1)
                
        #else:
            #region_counts['city'].append('NaN')
            #region_counts['country'].append('NaN')
            #region_counts['likes'].append(tweet['favorite_count'])
            #region_counts['tweets'].append(1)
            
        
            
        
    return region_counts
            


def count_cities(tweets):
    
    #in the user locations sometimes its Paris, France, sometimes France
    # sometimes Paris, here we try to filter this for cities
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    
    city_counts = {
    
    'city': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[0] in list_cities:
                city_counts['city'].append(list_loc[0])
                city_counts['likes'].append(tweet['favorite_count'])
                city_counts['tweets'].append(1)
                city_counts['retweets'].append(tweet['retweet_count'])
                city_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_cities:
            city_counts['city'].append(tweet['user_loc'])
            city_counts['likes'].append(tweet['favorite_count'])
            city_counts['tweets'].append(1)
            city_counts['retweets'].append(tweet['retweet_count'])
            city_counts['sentiment'].append(tweet['compound_score'])
            
            
            
    return city_counts


    
def count_countries(tweets):
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    country_counts = {
    
    'country': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[1] in list_countries:
                country_counts['country'].append(list_loc[1])
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])
                
            if list_loc[0] in list_cities:
                country_counts['country'].append(get_country(list_loc[0])) #translates city to country
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])

            
            
        elif tweet['user_loc'] in list_cities:
            country_counts['country'].append(get_country(tweet['user_loc']))
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_countries:
            country_counts['country'].append(tweet['user_loc'])
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
    return country_counts
    

In [149]:
country_counts = count_countries(data_dict)
df_country_counts = pd.DataFrame(country_counts)
df_country_counts.head()
#df_country_counts.shape

,country,tweets,likes,retweets,sentiment
0,Romania,1,3,0,0.4404
1,Romania,1,0,0,NaN
2,Bulgaria,1,0,0,0.2732
3,Greece,1,0,0,-0.3818
4,Poland,1,1,0,0.2415


In [153]:
import math

def clean_and_sum(df):
    
    df_country_counts = df_country_counts.dropna()
    df_country_counts = df_country_counts.groupby(by="country", as_index=False).sum()
    return df

def add_lat_lon(df):
    df['lat'] = df['country'].apply(region_lat)
    df['lon'] = df['country'].apply(region_lon)
    return df

def add_radius(df):

    df["radius"] = df["retweets"].apply(lambda likes: math.sqrt(likes)*1000 + 10000)
    return df

df_country_counts

,country,tweets,likes,retweets,sentiment,lat,lon,radius
0,Austria,11,5,88,1.2379,47.200000,13.200000,19380.831520
1,Belarus,1,0,0,0.2732,53.425061,27.697136,10000.000000
2,Belgium,10,0,18,0.9318,50.640281,4.666715,14242.640687
3,Bosnia,16,0,0,1.7878,44.305348,17.596147,10000.000000
4,Bulgaria,18,36,9,4.0560,42.607397,25.485662,13000.000000
5,Croatia,7,3,8,-2.0499,45.564344,17.011895,12828.427125
6,Czech Republic,71,273,200,10.0290,49.816700,15.474954,24142.135624
7,Denmark,87,1423,286,6.4096,55.670249,10.333328,26911.534525
8,Estonia,1,0,1,0.4939,58.752378,25.331908,11000.000000
9,Finland,158,492,1481,22.5007,63.246778,25.920916,48483.762810


In [ ]:
clean_and_sum()
add_lat_lon()
add_radius()

In [154]:
df_country_counts.to_csv('country_counts.csv')

In [110]:
city_counts = count_cities(data_dict)
df_city_counts = pd.DataFrame(city_counts)
df_city_counts.head()
#df_city_counts.shape

,city,tweets,likes,retweets
0,Bucharest,1,0,0
1,Bratislava,1,0,0
2,Thessaloniki,1,0,3
3,Bratislava,1,17,4
4,Zagreb,1,18,0


In [106]:
# drop rows with user_loc as city-name

#df_city_counts = df_city_counts[(df_city_counts != 'user_loc').all(axis=1)]
#df_city_counts = df_city_counts[(df_city_counts != 'NaN').all(axis=1)]
df_city_counts.head()

,city,tweets,likes,retweets


In [28]:
df_city_counts = df_city_counts.dropna()
df_city_counts = df_city_counts.groupby(by="city", as_index=False).sum()

df_city_counts['lat'] = df_city_counts['city'].apply(region_lat)
df_city_counts['lon'] = df_city_counts['city'].apply(region_lon)
#df_city_counts["radius"] = df_city_counts["tweets"].apply(lambda likes: math.sqrt(likes)*1000 + 10000)

df_city_counts

,city,tweets,likes,retweets,lat,lon
0,3 Rawlinson St,1,0,0,-27.469474,153.097644
1,Brussels,1,0,1,50.846557,4.351697
2,Earth,1,0,1,34.233137,-102.410749
3,Gironde,1,0,3,44.883746,-0.605126
4,Kampala,1,0,1,0.317714,32.581354
5,Kolkata,1,0,6,22.541418,88.357691
6,Limbo,1,625,221,5.224918,9.773734
7,Los Angeles,1,3432,871,34.053691,-118.242766
8,Nairobi,3,0,5,-1.303169,36.826061
9,Nanyuki,1,0,12,0.014030,37.074632


In [ ]:
df_city_counts.to_csv('city_counts.csv')


In [193]:
def sent_ref(df):
    df['sent_ref'] = df['sentiment'] / df['tweets']
    return df

def sent_color_green(sent):
    
    if sent > 0:
        green = 100
    else:
        green = 0
        
    return green


def sent_color_red(sent):
    
    if sent < 0:
        red = 100
    else:
        red = 0
    
    return red
    
    
def sent_color_blue(sent):
    blue = sent * 0
    return blue


def sent_shade(sent):
    if sent < 0:
        shade = -300 * sent + 50
    else:
        shade = 300 * sent + 50
    return shade



In [194]:
def sent_colors(df):
    df['r'] = df['sent_ref'].apply(sent_color_red)
    df['g'] = df['sent_ref'].apply(sent_color_green)
    df['b'] = df['sent_ref'].apply(sent_color_blue)
    df['s'] = df['sent_ref'].apply(sent_shade)
    df['sent_ref'] = df['sent_ref'].round(decimals=4)
    return df


In [195]:
 
df_country_counts.to_csv('country_counts.csv')
df_country_counts 

,country,tweets,likes,retweets,sentiment,lat,lon,radius,sent_ref,r,g,s,b
0,Austria,11,5,88,1.2379,47.200000,13.200000,19380.831520,0.1125,0,100,83.75,0.0
1,Belarus,1,0,0,0.2732,53.425061,27.697136,10000.000000,0.2732,0,100,131.96,0.0
2,Belgium,10,0,18,0.9318,50.640281,4.666715,14242.640687,0.0932,0,100,77.96,0.0
3,Bosnia,16,0,0,1.7878,44.305348,17.596147,10000.000000,0.1117,0,100,83.51,0.0
4,Bulgaria,18,36,9,4.0560,42.607397,25.485662,13000.000000,0.2253,0,100,117.59,0.0
5,Croatia,7,3,8,-2.0499,45.564344,17.011895,12828.427125,-0.2928,100,0,137.84,-0.0
6,Czech Republic,71,273,200,10.0290,49.816700,15.474954,24142.135624,0.1413,0,100,92.39,0.0
7,Denmark,87,1423,286,6.4096,55.670249,10.333328,26911.534525,0.0737,0,100,72.11,0.0
8,Estonia,1,0,1,0.4939,58.752378,25.331908,11000.000000,0.4939,0,100,198.17,0.0
9,Finland,158,492,1481,22.5007,63.246778,25.920916,48483.762810,0.1424,0,100,92.72,0.0


### streamlit

In [ ]:
map_data = pd.DataFrame(
    np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4],
    columns=['lat', 'lon'])
map_data

In [ ]:
cents = pd.read_csv('country_centroids.csv',delimiter=';')
cents = cents.dropna()



In [ ]:
cents_eur = pd.read_csv('cents_eur.csv',delimiter=';')


In [ ]:
def remove_dots(x):
    return int(str(x).replace(".", ""))

In [ ]:
def trunc(x):
    
    list_ =  ['.'.join( str(x).split('.')[0:2]),str(x).split('.')[2]]  
    merged = ''.join(list_)
   
    return merged



In [ ]:
#cents_eur['lat'] = cents_eur['lat'].apply(trunc)
#cents_eur['lon'] = cents_eur['lon'].apply(trunc)

In [ ]:
cents_eur.to_csv('cents_eur_trunc.csv')

## Database retrieve

In [102]:
####general data####


import requests

params=dict(q='',limit=100000)
headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
lexai_url = f"http://35.223.18.2/indexes/twitter_query/search/"
data = requests.get(lexai_url,params=params,headers=headers).json()
data_df=pd.DataFrame(data['hits'])
data_df = data_df.sort_values(by=['timestamp'])
data_dict = data_slice.to_dict('records')   #creates dictionary for further use
data_df.head()


,id,timestamp,user,text,text_en,date,lang,iso_lang,user_verified,followers_count,...,user_desc,user_desc_en,user_image,hashtags,mentions,retweet_count,favorite_count,link,compound_score,sentiment
17806,1397811490107838467,1.622092e+09,radosveta_vass,"1. @clubz_bg, Във връзка с нелепия материал, к...","1. @clubz_bg, In connection with the ridiculou...",2021/05/27 07:06:43,bg,bg,False,3714,...,Legal scholar & social advocate | Visiting Res...,Legal scholar & social advocate | Visiting Res...,http://pbs.twimg.com/profile_images/6427396249...,"КТБ, HumanRights",clubz_bg,5,10,https://twitter.com/radosveta_vass/status/1397...,-0.6996,negative
17805,1397815137743028227,1.622093e+09,hen02sng,RT @_nuthatch_: Widzę że Yixing się na węd...,RT @_nuthatch_: I see Yixing has switched to f...,2021/05/27 07:21:13,pl,pl,False,1990,...,exo nct ## love zone | khh🖇️krnb,exo nct ## love zone | khh🖇️krnb,http://pbs.twimg.com/profile_images/1399123298...,,_nuthatch_,15,0,https://twitter.com/hen02sng/status/1397815137...,0.0000,neutral
17804,1397815813164486657,1.622093e+09,TzvetVassilev,"RT @radosveta_vass: 1. @clubz_bg, Във връзка с...","RT @radosveta_vass: 1. @clubz_bg, In connectio...",2021/05/27 07:23:54,bg,bg,False,667,...,"Banking Chairman, Entrepreneur, Enemy of Bulga...","Banking Chairman, Entrepreneur, Enemy of Bulga...",http://pbs.twimg.com/profile_images/8091678486...,КТБ,"radosveta_vass, clubz_bg",5,0,https://twitter.com/TzvetVassilev/status/13978...,-0.5267,negative
17803,1397817049209053187,1.622093e+09,EYNCNrvu3JHzK82,RT @strani_placenik: Naprednjaci upravo tiho k...,RT @strani_placenik: Napredniks are just quiet...,2021/05/27 07:28:49,und,und,False,0,...,,None,http://pbs.twimg.com/profile_images/1397813907...,,strani_placenik,33,0,https://twitter.com/EYNCNrvu3JHzK82/status/139...,0.1280,neutral
17802,1397817744624656384,1.622093e+09,OPublicaINFO,“Livrabilele” programului “România educată” ...,The “deliverables” of the “Educated Romania” p...,2021/05/27 07:31:35,ro,ro,False,128,...,Publicatie Online - Revista presei,Online Publication - Press Review,http://pbs.twimg.com/profile_images/9060877700...,,,0,0,https://twitter.com/OPublicaINFO/status/139781...,0.6739,positive


In [197]:
#####press data#####

params=dict(q='',limit=100000)
headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
lexai_url = f"http://35.223.18.2/indexes/twitter_press/search/"
data_press = requests.get(lexai_url,params=params,headers=headers).json()
data_press_df=pd.DataFrame(data_press['hits'])
data_press_df = data_press_df.sort_values(by=['timestamp'])
data_press_df = data_press_df[data_press_df["timestamp"] >= 1.622115e+09]
data_press_df.head()
data_press_df.shape

(4590, 22)

In [198]:
###politician data####

params=dict(q='energy',limit=100000)
headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
lexai_url = f"http://35.223.18.2/indexes/twitter_politicians/search/"
data_pol = requests.get(lexai_url,params=params,headers=headers).json()
data_pol_df=pd.DataFrame(data_pol['hits'])
data_pol_df = data_pol_df.sort_values(by=['timestamp'])
data_pol_df = data_pol_df[data_pol_df["timestamp"] >= 1.622115e+09]
data_pol_df.head()
data_pol_df.shape

(173, 25)

In [13]:
data_slice = data_slice.to_dict('records')